In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

%matplotlib inline

# ResNet Model

## Residual Block

In [2]:
class ResBlock(nn.Module):
    def __init__(self, channels_in, channels_out, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels_in, channels_out, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channels_out)
        self.conv2 = nn.Conv2d(channels_out, channels_out, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channels_out)
        
        self.shortcut = None
        if channels_in != channels_out:
            self.shortcut = nn.Sequential(
                nn.Conv2d(channels_in, channels_out, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channels_out),
            )
    
    def forward(self, x):
        shct = x
        if self.shortcut is not None: shct = self.shortcut(shct)
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x += shct
        x = F.relu(x)
        return x

## ResNet Class

In [3]:
class ResNetShallow(nn.Module):
    def __init__(self, first_channels, num_blocks, num_classes):
        super(ResNetShallow, self).__init__()
        self.channels = first_channels
        
        self.conv1 = nn.Conv2d(3, self.channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(self.channels)
        self.layer1 = self._make_layer(num_blocks[0], False)
        self.layer2 = self._make_layer(num_blocks[1], True)
        self.layer3 = self._make_layer(num_blocks[2], True)
        self.layer4 = self._make_layer(num_blocks[3], True)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.dropout = nn.Dropout(0.4)
        self.fc = nn.Linear(self.channels, num_classes)
        
    def _make_layer(self, num_blocks, downsample=True):
        layers = []
        for i in range(num_blocks):
            if i==0 and downsample:
                layers.append(ResBlock(self.channels, self.channels*2, 2))
                self.channels *= 2
            else:
                layers.append(ResBlock(self.channels, self.channels, 1))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        
        x = self.dropout(x)
        x = self.fc(x)
        
        return x

In [4]:
def ResNet18(first_channels):
    return ResNetShallow(first_channels, [2, 2, 2, 2], 10)

def ResNet34(first_channels):
    return ResNetShallow(first_channels, [3, 4, 6, 3], 10)

# Dataset

In [5]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=2),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.5),
    transforms.RandomAffine(degrees=10., scale=(0.9,1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.4914009 , 0.48215896, 0.4465308), (0.24703279, 0.24348423, 0.26158753))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914009 , 0.48215896, 0.4465308), (0.24703279, 0.24348423, 0.26158753))
])

In [6]:
trainset = torchvision.datasets.CIFAR10(
    root='../data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(
    root='../data', train=False, download=True, transform=transform_test)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
num_train = trainset.__len__()
num_test = testset.__len__()
bs_train = 1024
bs_test = 2048

In [8]:
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=bs_train, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=bs_test, shuffle=True, num_workers=4)

In [9]:
classes = ('plane', 'car', 'bird', 'car', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Training

In [10]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [11]:
net = ResNet18(64)
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [12]:
epochs=50
# num_triangle = 1

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.08, momentum=0.9, weight_decay=5e-4)
# scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.001, max_lr=1.0,
#                                               step_size_up=int(epochs*len(trainloader)/2/num_triangle), mode='triangular2')
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, 0.8, epochs=epochs, steps_per_epoch=len(trainloader),
                                                anneal_strategy='cos')

In [13]:
print(f"Begin Training for {epochs} epoch")
test_best_acc = 1000.
for epoch in range(1, epochs + 1):
    train_loss = 0
    train_correct = 0
    train_total = 0
    
    net.train()
    for i, (x, y) in enumerate(trainloader):
        x, y = x.to(device), y.to(device)
        
        pred = net(x)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        preds = torch.argmax(pred, dim=1)
        train_loss += loss.item()
        train_correct += (preds == y).sum().item()
        train_total += y.size(0)
        
    test_loss = 0
    test_correct = 0
    test_total = 0
    
    net.eval()
    for x, y in testloader:
        x = x.to(device)
        y = y.to(device)

        with torch.no_grad():
            pred = net(x)
            loss = criterion(pred, y)

            preds = torch.argmax(pred, dim=1)
            test_loss += loss.item()
            test_correct += (preds == y).sum().item()
            test_total += y.size(0)

    print(f'Epoch: {epoch} || train_loss: {train_loss / ((num_train - 1) // bs_train + 1):.5f} || train_acc : {train_correct / train_total:.5f} || test_loss: {test_loss / ((num_test - 1) // bs_test + 1):.5f} || test_acc : {test_correct / test_total:.5f} ')
    
    if test_best_acc > test_correct / test_total:
        torch.save(net.state_dict(), 'models/CIFAR10/ResNet18/model.pt')
        
torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss / ((num_train - 1) // bs_train + 1),
            }, "models/CIFAR10/ResNet18/last_checkpoints.pt")

Begin Training for 50 epoch
Epoch: 1 || train_loss: 1.85062 || train_acc : 0.31466 || test_loss: 1.98400 || test_acc : 0.36220 
Epoch: 2 || train_loss: 1.46473 || train_acc : 0.46522 || test_loss: 1.52801 || test_acc : 0.48030 
Epoch: 3 || train_loss: 1.25937 || train_acc : 0.54566 || test_loss: 1.40796 || test_acc : 0.54170 
Epoch: 4 || train_loss: 1.04914 || train_acc : 0.62850 || test_loss: 1.23037 || test_acc : 0.61740 
Epoch: 5 || train_loss: 0.90766 || train_acc : 0.68386 || test_loss: 1.04192 || test_acc : 0.64890 
Epoch: 6 || train_loss: 0.80087 || train_acc : 0.72196 || test_loss: 0.81511 || test_acc : 0.71370 
Epoch: 7 || train_loss: 0.72884 || train_acc : 0.74816 || test_loss: 0.66504 || test_acc : 0.76440 
Epoch: 8 || train_loss: 0.65671 || train_acc : 0.77190 || test_loss: 0.62871 || test_acc : 0.77390 
Epoch: 9 || train_loss: 0.62561 || train_acc : 0.78568 || test_loss: 0.63788 || test_acc : 0.77410 
Epoch: 10 || train_loss: 0.58473 || train_acc : 0.79988 || test_loss: 0.